In [ ]:
nodes = ["Eve","Cain", "Seth", "Enos", "Noam", "Abel", "Awan", "Enoch", "Azura"]
parents = ["", "Eve", "Eve", "Seth", "Seth", "Eve", "Eve", "Awan", "Eve"]
parents

In [ ]:
import re
import json
from bson import json_util
 
date_format = '%Y-%m-%d %H:%M:%S'
 
def replace_matched(match):
    groups = match.groups()
    return f'"{next(filter(None, groups), "")}"'
 
def add_comma_between_objects(data):
    # Split the data into individual blocks and check if a comma is present after each block
    blocks = data.strip().split('\n}\n')
    formatted_data = ''
    for i, block in enumerate(blocks):
        if block.endswith(',') or i == len(blocks) - 1:
            formatted_data += block
        else:
            formatted_data += f"{block}\n}},"
        # Add a newline except for the last block
        if i != len(blocks) - 1:
            formatted_data += '\n'
 
    return formatted_data
 
def read_mongoextjson_file(filename):
    with open(filename, "r") as f:
        bsondata = f.read()
        bsondata = add_comma_between_objects(bsondata)
        pattern = re.compile(r'(ObjectId|ISODate|Double|NumberInt)\s*\(\s*(\"?\S+\"?)\s*\)')
        replaced_data = pattern.sub(replace_matched, bsondata)
        data = json.loads(f"[{replaced_data}]", object_hook=json_util.object_hook)
        return data

In [ ]:
import pandas as pd
# read process guide from ../data/prod-mnr/process_guide.json in data frame
df = pd.DataFrame()
pg = read_mongoextjson_file("../data/prod-mnr/process_guide.json")
pes = pd.DataFrame(pg)["process_elements"]
for pe in pes:
    # print(pe)
    pe = pd.DataFrame(pe)
    pe["parent_name"] = pe["parent_id"].map(pe.set_index('id')['name'])
    # append pe to df
    df = df.append(pe[["name","parent_name"]])
# pe = pd.DataFrame(pg["process_elements"])
# remove duplicates in df
df = df.drop_duplicates()
# replace Nan with empty string
df = df.fillna("Root")
df

In [ ]:
import pandas as pd
# read process guide from ../data/prod-mnr/process_guide.json in data frame

pg = read_mongoextjson_file("../data/prod-mnr/process_guide.json")
pe = pd.DataFrame(pg[3]["process_elements"])
pe.columns


# add a column for parent_name and fill it with the parent name
pe["parent_name"] = pe["parent_id"].map(pe.set_index('id')['name'])
pe

In [ ]:
def addParent(x):
    # x["parent_name"] = "Hello"
    y = pd.DataFrame(x)
    x["parent_name"] = y["parent_id"].map(y.set_index('id')['name'])
    return y
df = pd.DataFrame(pg)
a = pd.DataFrame(df["process_elements"][0])

a.apply(addParent, axis=1)


In [ ]:
df = pd.DataFrame(pg)
pd.json_normalize(df["process_elements"])

In [ ]:
df = pd.DataFrame(pg)
x = pd.DataFrame(df["process_elements"])
# explode the process_elements column
x = x.explode("process_elements", ignore_index=False)
y = pd.DataFrame(x["process_elements"])
pe
# x["process_elements"]
pe = pd.json_normalize(x["process_elements"]).reset_index()
# pe[["name","parent_id"]]
pedf = pe[["index", "id", "name","parent_id"]]
# pedf["parent_name"] = pe["parent_id"].map(pedf.set_index('id')['name'])


In [ ]:
nodes = df[["name"]].values.tolist()
parents = df[["parent_name"]].values.tolist()
print(nodes)
print(parents)

In [ ]:
import plotly.express as px

print(nodes)
print(parents)
fig = px.treemap(
    names = nodes[:15],
    parents = parents[:15]
)
fig.update_traces(root_color="lightgrey")
fig.update_layout(margin = dict(t=50, l=25, r=25, b=25))
fig.show()
